## Import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import math
import os
from skimage import measure
from skimage.measure import regionprops, regionprops_table
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.preprocessing.image import load_img
from importlib import reload
import segmenteverygrain as seg
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
from tqdm import trange
%matplotlib qt

## Load models

In [2]:
model = seg.Unet()
model.compile(optimizer=Adam(), loss=seg.weighted_crossentropy, metrics=["accuracy"])
model.load_weights('./beard_and_weyl_model/seg_model') # replace this if you have a finetuned Unet model and want to use it

# the SAM model checkpoints can be downloaded from: https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
sam = sam_model_registry["default"](checkpoint="C:/Users/owach/OneDrive/School/Research/grain-segmentations/sam_vit_h_4b8939.pth")

## Run segmentation

Grains are supposed to be well defined in the image; e.g., if a grain consists of only a few pixels, it is unlikely to be detected.

The segmentation can take a few minutes even for medium-sized images, so do not start with large images (downsample them if necessary). Images with ~2000 pixels along their largest dimension are a good start.

If you have a much larger image, see the section **"Run segmentation on large image"** at the end of the notebook.

Image used below is available from [here](https://github.com/zsylvester/segmenteverygrain/blob/main/torrey_pines_beach.jpeg).

In [10]:
import numpy as np
from keras.preprocessing.image import load_img

# Replace this with the path to your image:
fname = 'C:/Users/owach/OneDrive/School/Research/images/circularityimages/.97_inverted.png'

big_im = np.array(load_img(fname))
big_im_pred = seg.predict_big_image(big_im, model, I=256)
print(big_im)



segmenting image tiles...


100%|██████████| 4/4 [00:00<00:00,  4.88it/s]


[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]]


In [11]:
# # decreasing the 'dbs_max_dist' parameter results in more SAM prompts (and longer processing times):
reload(seg)
big_im = np.array(load_img(fname))
big_im_pred = seg.predict_big_image(big_im, model, I=256)

# decreasing the 'dbs_max_dist' parameter results in more SAM prompts (and longer processing times):
labels, coords = seg.label_grains(big_im, big_im_pred, dbs_max_dist=20.0) # Unet prediction

# SAM segmentation, using the point prompts from the Unet:
all_grains, labels, mask_all, grain_data, fig, ax = seg.sam_segmentation(sam, big_im, big_im_pred, 
                                                coords, labels, min_area=400.0, plot_image=True, remove_edge_grains=False, remove_large_objects=False)

segmenting image tiles...


100%|██████████| 4/4 [00:00<00:00,  4.50it/s]


creating masks using SAM...


100%|██████████| 2/2 [00:00<00:00,  2.49it/s]


finding overlapping polygons...


2it [00:00, 20.66it/s]


finding best polygons...


100%|██████████| 1/1 [00:00<00:00,  6.73it/s]


creating labeled image...


100%|██████████| 1/1 [00:00<00:00, 11.25it/s]


Use this figure to check the distribution of SAM prompts (= black dots):

In [12]:
plt.figure(figsize=(15,10))
plt.imshow(big_im_pred)
plt.scatter(np.array(coords)[:,0], np.array(coords)[:,1], c='k')
plt.xticks([])
plt.yticks([]);

## Delete or merge grains in segmentation result
* click on the grain that you want to remove and press the 'x' key
* click on two grains that you want to merge and press the 'm' key (they have to be the last two grains you clicked on)
* press the 'g' key to hide the grain masks (so that you can see the original image better); press the 'g' key again to show the grain masks

In [14]:
grain_inds = []
cid1 = fig.canvas.mpl_connect('button_press_event', 
                              lambda event: seg.onclick2(event, all_grains, grain_inds, ax=ax))
cid2 = fig.canvas.mpl_connect('key_press_event', 
                              lambda event: seg.onpress2(event, all_grains, grain_inds, fig=fig, ax=ax))

Traceback (most recent call last):
  File "c:\Users\owach\anaconda3\envs\grain-segmentations\lib\site-packages\matplotlib\cbook\__init__.py", line 304, in process
    func(*args, **kwargs)
  File "C:\Users\owach\AppData\Local\Temp\ipykernel_28464\32070719.py", line 5, in <lambda>
    lambda event: seg.onpress2(event, all_grains, grain_inds, fig=fig, ax=ax))
  File "c:\Users\owach\OneDrive\School\Research\grain_analyzer\segmenteverygrain\segmenteverygrain.py", line 906, in onpress2
    ax.patches[grain_inds[-1]].remove()
IndexError: list index out of range
Traceback (most recent call last):
  File "c:\Users\owach\anaconda3\envs\grain-segmentations\lib\site-packages\matplotlib\cbook\__init__.py", line 304, in process
    func(*args, **kwargs)
  File "C:\Users\owach\AppData\Local\Temp\ipykernel_28464\32070719.py", line 5, in <lambda>
    lambda event: seg.onpress2(event, all_grains, grain_inds, fig=fig, ax=ax))
  File "c:\Users\owach\OneDrive\School\Research\grain_analyzer\segmenteverygra

Run this cell if you do not want to delete / merge existing grains anymore; it is a good idea to do this before moving on to the next step.

In [15]:
fig.canvas.mpl_disconnect(cid1)
fig.canvas.mpl_disconnect(cid2)

Use this function to update the 'all_grains' list after deleting and merging grains:

In [92]:
all_grains, labels, mask_all, fig, ax = seg.get_grains_from_patches(ax, big_im)

100%|██████████| 187/187 [00:00<00:00, 237.68it/s]


Plot the updated set of grains:

In [93]:
fig, ax = plt.subplots(figsize=(15,10))
ax.imshow(big_im)
plt.xticks([])
plt.yticks([])
seg.plot_image_w_colorful_grains(big_im, all_grains, ax, cmap='Paired')
# seg.plot_grain_axes_and_centroids(all_grains, labels, ax, linewidth=1, markersize=10)
plt.xlim([0, np.shape(big_im)[1]])
plt.ylim([np.shape(big_im)[0], 0]);

100%|██████████| 187/187 [00:00<00:00, 230.90it/s]


## Add new grains using the Segment Anything Model

* click on unsegmented grain that you want to add
* press the 'x' key if you want to delete the last grain you added
* press the 'm' key if you want to merge the last two grains that you added
* right click outside the grain (but inside the most recent mask) if you want to restrict the grain to a smaller mask - this adds a background prompt

In [13]:
predictor = SamPredictor(sam)
predictor.set_image(big_im) # this can take a while
coords = []
cid3 = fig.canvas.mpl_connect('button_press_event', lambda event: seg.onclick(event, ax, coords, big_im, predictor))
cid4 = fig.canvas.mpl_connect('key_press_event', lambda event: seg.onpress(event, ax, fig))

In [16]:
fig.canvas.mpl_disconnect(cid3)
fig.canvas.mpl_disconnect(cid4)

In [17]:
reload(seg)

<module 'segmenteverygrain' from 'c:\\Users\\owach\\OneDrive\\School\\Research\\grain_analyzer\\segmenteverygrain\\segmenteverygrain.py'>

After you are done with the deletion / addition of grain masks, run this cell to generate an updated set of grains:

In [18]:
all_grains, labels, mask_all, fig, ax = seg.get_grains_from_patches(ax, big_im)

100%|██████████| 5/5 [00:00<00:00, 246.46it/s]


## Get grain size distribution

Run this cell and then click (left mouse button) on one end of the scale bar in the image and click (right mouse button) on the other end of the scale bar:

In [19]:
from keras.preprocessing.image import load_img, img_to_array
scale = 'C:/Users/owach/Downloads/Inverted_Images_Hackathon/scale.png'
# Load the image and convert it to a NumPy array
scale_im = img_to_array(load_img(scale))

# Create a figure and axis
fig, ax = plt.subplots()

# Display the image
ax.imshow(scale_im.astype(np.uint8))  # Ensure the image data is in the correct type for display
ax.axis('off')  # Optional: Hide the axes

# Show the plot
plt.show()

In [16]:
cid5 = fig.canvas.mpl_connect('button_press_event', lambda event: seg.click_for_scale(event, ax))


number of pixels: 131.35


Use the length of the scale bar in pixels (it should be printed above) to get the scale of the image (in units / pixel):

In [20]:
n_of_units = 500
units_per_pixel = n_of_units/131.35 # length of scale bar in pixels

In [14]:
import math

In [21]:
print("Labels shape:", labels.shape)
print("Intensity image shape:", big_im.shape)


Labels shape: (436, 771)
Intensity image shape: (436, 771, 3)


In [22]:
props = regionprops_table(labels.astype('int'), intensity_image = big_im, properties =\
        ('label', 'area', 'centroid', 'major_axis_length', 'minor_axis_length', 
         'orientation', 'perimeter', 'max_intensity', 'mean_intensity', 'min_intensity'))
grain_data = pd.DataFrame(props)
grain_data['major_axis_length'] = grain_data['major_axis_length'].values*units_per_pixel
grain_data['minor_axis_length'] = grain_data['minor_axis_length'].values*units_per_pixel
grain_data['perimeter'] = grain_data['perimeter'].values*units_per_pixel
grain_data['area'] = grain_data['area'].values*units_per_pixel**2

In [23]:
grain_data.head(10)

,label,area,centroid-0,centroid-1,major_axis_length,minor_axis_length,orientation,perimeter,max_intensity-0,max_intensity-1,max_intensity-2,mean_intensity-0,mean_intensity-1,mean_intensity-2,min_intensity-0,min_intensity-1,min_intensity-2
0,1,11577.815746,33.912390,134.440551,125.836198,117.381975,0.589572,394.899564,235.0,235.0,235.0,18.568210,18.568210,18.568210,0.0,0.0,0.0
1,2,11751.700338,35.125771,176.641184,125.950653,118.982614,-0.025035,398.435659,238.0,238.0,238.0,24.187423,24.187423,24.187423,0.0,0.0,0.0
2,3,11911.094547,35.231144,217.890511,129.474833,117.497276,0.214450,402.895396,250.0,250.0,250.0,24.212895,24.212895,24.212895,0.0,0.0,0.0
3,4,9186.902607,35.634069,256.623028,111.638743,105.395940,0.785371,360.369424,246.0,246.0,246.0,25.891167,25.891167,25.891167,0.0,0.0,0.0
4,5,7679.902810,34.528302,98.433962,104.592782,93.550189,-0.189758,320.455905,250.0,250.0,250.0,56.867925,56.867925,56.867925,1.0,1.0,1.0


In [24]:
# Assuming grain_data is a DataFrame and units_per_pixel is defined:
import numpy as np

# Calculate roundness
grain_data['roundness'] = (4 * (grain_data['area'].values * units_per_pixel**2)) / (np.pi * (grain_data['major_axis_length'].values * units_per_pixel)**2)

# Calculate circularity
grain_data['circularity'] = (4 * np.pi * (grain_data['area'].values * units_per_pixel**2)) / ((grain_data['perimeter'].values * units_per_pixel)**2)



In [25]:
grain_data = grain_data[np.isfinite(grain_data['roundness']) & np.isfinite(grain_data['circularity'])]

In [26]:
grain_data.shape

(5, 19)

In [27]:
grain_data.head(8)

,label,area,centroid-0,centroid-1,major_axis_length,minor_axis_length,orientation,perimeter,max_intensity-0,max_intensity-1,max_intensity-2,mean_intensity-0,mean_intensity-1,mean_intensity-2,min_intensity-0,min_intensity-1,min_intensity-2,roundness,circularity
0,1,11577.815746,33.912390,134.440551,125.836198,117.381975,0.589572,394.899564,235.0,235.0,235.0,18.568210,18.568210,18.568210,0.0,0.0,0.0,0.930948,0.932960
1,2,11751.700338,35.125771,176.641184,125.950653,118.982614,-0.025035,398.435659,238.0,238.0,238.0,24.187423,24.187423,24.187423,0.0,0.0,0.0,0.943213,0.930238
2,3,11911.094547,35.231144,217.890511,129.474833,117.497276,0.214450,402.895396,250.0,250.0,250.0,24.212895,24.212895,24.212895,0.0,0.0,0.0,0.904672,0.922098
3,4,9186.902607,35.634069,256.623028,111.638743,105.395940,0.785371,360.369424,246.0,246.0,246.0,25.891167,25.891167,25.891167,0.0,0.0,0.0,0.938533,0.888962
4,5,7679.902810,34.528302,98.433962,104.592782,93.550189,-0.189758,320.455905,250.0,250.0,250.0,56.867925,56.867925,56.867925,1.0,1.0,1.0,0.893845,0.939786


In [28]:
import os
output_path = 'C:/Users/owach/OneDrive/School/Research/grain_analyzer/Outputted_Dataframes/'
file_name_without_extension = os.path.splitext(os.path.basename(fname))[0]

print(file_name_without_extension)  # Output: image

.97_inverted


In [23]:
#grain_data.to_csv('./Outputted_Dataframes' + fname[:-4]+'.csv') # save grain data to CSV file
grain_data.to_csv(output_path + file_name_without_extension+ '1.csv')

In [24]:
# plot histogram of grain axis lengths
plt.figure()
plt.hist(grain_data['major_axis_length'], np.arange(0, 100, 1), alpha=0.5)
plt.hist(grain_data['minor_axis_length'], np.arange(0, 100, 1), alpha=0.5)
plt.xlim(0,100)
plt.xlabel('axis length (microns)')
plt.ylabel('count');

In [26]:
import numpy as np
import matplotlib.pyplot as plt

# Sample data: replace this with your actual roundness values
roundness_values = grain_data['roundness'].values 

# Define the bins for the histogram
bins = [0, 0.15, 0.2, 0.3, 0.4, 0.6, 1]  #defined by Beard and Weyl (1973)
labels = ['Very Angular', 'Angular', 'Sub-Angular', 'Sub-Rounded', 'Rounded', 'Well Rounded']

# Count frequencies in each bin
hist, bin_edges = np.histogram(roundness_values, bins=bins)

# Create the histogram plot
plt.figure(figsize=(10, 6))
plt.bar(labels, hist, width=0.6, color='skyblue', edgecolor='black')
plt.xlabel('Grain Roundness Categories')
plt.ylabel('Frequency')
plt.title('Histogram of Grain Roundness')
plt.xticks(rotation=15)
plt.grid(axis='y')
plt.savefig('C:/Users/owach/OneDrive/School/Research/grain_analyzer/Outputted_Images/' + file_name_without_extension + 'roundness_histogram.png', dpi=300, bbox_inches='tight')
# Show the plot
plt.show()


In [31]:
circularity_stats = grain_data['circularity'].describe()

# If you want additional statistics like variance and skewness
variance = grain_data['circularity'].var()
skewness = grain_data['circularity'].skew()

# Print statistics
print("Basic Statistics for circularity Values:")
print(circularity_stats)
print(f"\nVariance: {variance}")
print(f"Skewness: {skewness}")


KeyError: 'sphericity'

## Save mask and grain labels to PNG files

In [50]:
dirname = 'C:/Users/owach/OneDrive/School/Research/grain-segmentations/images/'
# write grayscale mask to PNG file
cv2.imwrite(dirname + fname.split('/')[-1][:-4] + '_mask.png', mask_all)
# Save the image as a PNG file
cv2.imwrite(dirname + fname.split('/')[-1][:-4] + '_image.png', cv2.cvtColor(big_im, cv2.COLOR_BGR2RGB))

True

## Run segmentation on large image (new!)
In this case 'fname' points to an image that is larger than a few megapixels and has thousands of grains.
The 'predict_large_image' function breaks the input image into smaller patches and it runs the segmentation process on each patch.

In [33]:
reload(seg)
from PIL import Image
Image.MAX_IMAGE_PIXELS = None # needed if working with very large images
fname = "/Users/zoltan/Dropbox/Segmentation/images/mair_et_al_L2_DJI_0382_image.png"
all_grains = seg.predict_large_image(fname, model, sam, min_area=400.0, patch_size=2000, overlap=200)

segmenting image tiles...


100%|██████████████████████████████████████████████████████████████████████████████████| 8/8 [00:06<00:00,  1.30it/s]


creating masks using SAM...


100%|████████████████████████████████████████████████████████████████████████████| 3197/3197 [03:25<00:00, 15.54it/s]


finding overlapping polygons...


...

1712it [00:06, 257.94it/s]


finding best polygons...


100%|██████████████████████████████████████████████████████████████████████████████| 508/508 [00:06<00:00, 74.57it/s]


creating labeled image...


4731it [00:02, 2242.65it/s]
100%|█████████████████████████████████████████████████████████████████████████████| 319/319 [00:00<00:00, 881.09it/s]


In [34]:
# plot results
big_im = np.array(load_img(fname))
fig, ax = plt.subplots(figsize=(15,10))
ax.imshow(big_im)
plt.xticks([])
plt.yticks([])
seg.plot_image_w_colorful_grains(big_im, all_grains, ax, cmap='Paired')
plt.axis('equal')
plt.xlim([0, np.shape(big_im)[1]])
plt.ylim([np.shape(big_im)[0], 0]);

In [40]:
# this is a faster way of deleting false positives (because it avoids highlighting and deleting the grains)
grain_inds = []
cid1 = fig.canvas.mpl_connect('button_press_event', 
                              lambda event: seg.onclick2(event, all_grains, grain_inds, ax=ax, select_only=True))

In [42]:
# delete polygons from 'all_grains'
grain_inds = np.unique(grain_inds)
grain_inds = sorted(grain_inds, reverse=True)
for ind in grain_inds:
    all_grains.remove(all_grains[ind])

In [ ]:
fig.canvas.mpl_disconnect(cid1)

In [ ]:
# plot image with updated grains
big_im = np.array(load_img(fname))
fig, ax = plt.subplots(figsize=(15,10))
ax.imshow(big_im)
plt.xticks([])
plt.yticks([])
seg.plot_image_w_colorful_grains(big_im, all_grains, ax, cmap='Paired')
plt.axis('equal')
plt.xlim([0, np.shape(big_im)[1]])
plt.ylim([np.shape(big_im)[0], 0]);

In [ ]:
# collect data from plot, including mask
all_grains, labels, mask_all, fig, ax = seg.get_grains_from_patches(ax, big_im)

In [140]:
from skimage.measure import regionprops, regionprops_table
props = regionprops_table(labels, intensity_image = big_im, properties=('label', 'area', 'centroid', 'major_axis_length', 'minor_axis_length', 
                                                                                'orientation', 'perimeter', 'max_intensity', 'mean_intensity', 'min_intensity'))

In [142]:
df = pd.DataFrame(props)

In [143]:
df.head()

,label,area,centroid-0,centroid-1,major_axis_length,minor_axis_length,orientation,perimeter,max_intensity-0,max_intensity-1,max_intensity-2,mean_intensity-0,mean_intensity-1,mean_intensity-2,min_intensity-0,min_intensity-1,min_intensity-2
0,1,830.0,43960.320482,1677.862651,38.752309,27.316355,0.273334,107.497475,200.0,205.0,214.0,77.914458,67.026506,61.689157,10.0,25.0,20.0
1,2,1136.0,43590.179577,686.280810,43.211267,34.025389,0.857311,126.225397,207.0,218.0,211.0,145.168134,143.100352,143.987676,34.0,37.0,65.0
2,3,1571.0,43189.798218,963.990452,56.501983,35.768381,0.805265,149.923882,217.0,222.0,216.0,149.928708,145.283896,141.232336,26.0,31.0,21.0
3,4,3388.0,43150.272137,588.850945,92.152532,47.986861,1.432245,237.823376,226.0,224.0,226.0,144.553424,139.338843,133.357143,32.0,41.0,39.0
4,5,897.0,42856.816054,620.888517,42.509864,27.112746,-1.446275,114.568542,223.0,223.0,216.0,177.035674,176.288740,176.924192,68.0,59.0,67.0


In [195]:
# plot grains in black and white
plt.figure()
# plt.imshow(big_im[0:2001, 0:2001, :])
for grain in tqdm(all_grains):
    plt.plot(grain.exterior.xy[0], grain.exterior.xy[1], 'k', linewidth=0.2)
plt.axis('equal')
plt.gca().invert_yaxis();

100%|███████████████████████████████████| 78923/78923 [00:21<00:00, 3616.88it/s]


After plotting the results, you will want to use the functions for deleting, merging, and adding grains (see above), before saving the results (same workflow as for a small image).